# [Transit Engine](https://developer.hashicorp.com/vault/api-docs/secret/transit)

In [ ]:
%env  WORKDIR=/tmp/vault    

In [ ]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

In [ ]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-p5fh.josemerchan-313707.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.GqTsfXwqgl0bK86WbGIfCBZz

## Enable Transit Engine

In [ ]:
! vault secrets enable transit

## Encryption

### Create keys for encryption

In [ ]:
! vault write -f transit/keys/orders
# Tipos de kes soportados: https://developer.hashicorp.com/vault/api-docs/secret/transit#type

### Encrypt/Decrypt

In [ ]:
%%bash
# ENCRYPT
vault write transit/encrypt/orders plaintext=$(base64 <<< "4111 1111 1111 1111") -format=json | jq -r .data.ciphertext

export CIPHERTEXT=$(vault write transit/encrypt/orders plaintext=$(base64 <<< "4111 1111 1111 1111") -format=json | jq -r .data.ciphertext)

# DECRYPT
vault write transit/decrypt/orders ciphertext=$CIPHERTEXT -format=json | jq -r .data.plaintext | base64 -d


## Signing

### Create keys

In [ ]:
! vault write -f  transit/keys/signature type=rsa-4096

In [ ]:
%%bash
INPUT=$(echo -n "un string no muy largo" | base64)
# Operación de firma
SIGNATURE=$(vault write -format=json transit/sign/signature input=$INPUT hash_algorithm=sha2-256 -format=json | jq -r .data.signature)
echo $SIGNATURE

# Verificación de firma
vault write transit/verify/signature input=$INPUT signature=$SIGNATURE  hash_algorithm=sha2-256

## HMAC

### Create keys

In [ ]:
! vault write -f  transit/keys/hmacker type=hmac key_size=512

In [ ]:
%%bash
INPUT=$(echo -n "otro string largo, pero que muy largo...... ... .. " | base64)
# Operación de hmac
HMAC=$(vault write -format="json" transit/hmac/hmacker algorithm="sha2-512" input=$INPUT  | jq -r .data.hmac)
echo $HMAC

# Verificación de hmac
vault write transit/verify/hmacker input=$INPUT algorithm="sha2-512" hmac=$HMAC

## HASH

In [ ]:
%%bash
INPUT=$( echo -n "otro string largo pero que muy largo 32323232323" | base64)
# Operación de hash
HASH=$(vault write -format=json transit/hash input=$INPUT algorithm=sha3-512 format=base64 -format=json | jq -r .data.sum)
echo $HASH

# Clean UP

In [ ]:
! vault secrets disable transit